In [6]:
import urllib2
import pandas as pd
wildcards = []
search_urls = []
CIK = []
Fund_Name = []
Vowels = ['a','e','i','o','u']
for i in Vowels:
    wildcards.append(i + "*")

In [7]:
for item in wildcards:  
    url = 'https://www.sec.gov/cgi-bin/series?company='+item+'&CIK=&start=1&count=500'
    html = urllib2.urlopen(url).read()
    
    last_page = False
    
    counter = 0  #only use this if you want to stop at certain page.
    
    while last_page == False:
    
        index = html.find("<table summary=") #going closer to the results table
        html2 = html[index:]

        #SAME CODE AS BEFORE STARTS HERE
        while html2.find("a class=\"search\" href=\"/cgi-bin/browse-edgar?CIK=") != -1: 

            #We are running a while loop because we want to get everything in the page
            index = html2.find("a class=\"search\" href=\"/cgi-bin/browse-edgar?CIK=")
            remaining = html2[index:]

            start = remaining.find(">")
            end = remaining.find("<")
            CIK.append(remaining[start+1:end])

            remaining = remaining[end:]
            index = remaining.find("</td>")
            remaining = remaining[index:]

            index = remaining.find("action=getcompany&amp;scd=series")
            remaining = remaining[index:]

            start = remaining.find(">")
            end = remaining.find("<")
            Fund_Name.append(remaining[start+1:end].replace(" ", "_"))

            remaining = remaining[end:]
            index = remaining.find("</td>")
            html2 = remaining[index:]
        #SAME CODE AS BEFORE ENDS
        
        index = html.find('count=500">[NEXT]')
        
        if index == -1:
            last_page == True
        
        else:
        
            html2 = html[index-20:index+100]
            start=html2.find('start=')+6
            end = html2.find('&amp;count=')
            start_number = html2[start:end]

            url = 'https://www.sec.gov/cgi-bin/series?company='+item+'&CIK=&start='+start_number+'&count=500'

            html = urllib2.urlopen(url).read()
        
        counter += 1
        if counter == 10:
            last_page = True    #only use this to limit the number of pages to go through.  You can drop this later.
    

In [8]:
outputdf=pd.DataFrame(CIK, columns=['CIK'])
outputdf['Fund_Name']=Fund_Name
outputdf.to_csv('allmf.csv')

In [10]:
import time
from urllib import FancyURLopener  # This is library that helps us create the headless browser
from random import choice #This library helps pick a random item from a list

user_agents = [
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
]

In [15]:

CIK_13F = []
weblinks = []

name = []

In [16]:
for item in CIK:
    
    link = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=' + item + '&type=13F-HR&dateb=&count=100&scd=filings'
    
    class MyOpener(FancyURLopener, object):
        version = choice(user_agents)
    
    myopener = MyOpener()
    page=myopener.open(link)
    
    html = page.read()
    
    # In the above codes, we are creating a class called MyOpener, which takes in a random input from the user agent list
    # and then opens the link in the supposed browser that it is mimicking and reads the html code. For every link, there is
    # a random choice of browser from the 5 that is listed
            
    index = html.find('a href=\"/Archives/edgar/data/')
    html2 = html[index:]

    while html2.find('a href=\"/Archives/edgar/data/') != -1:

        start = html2.find('a href=\"/Archives/edgar/data/')
        remaining = html2[start:]
        end = remaining.find('id=\"documentsbutton\"')
        weblinks.append('https://www.sec.gov'+remaining[8:end-2])
        remaining = remaining[end:]

        index = remaining.find('</td>')
        html2 = remaining[index:]

        CIK_13F.append(item)
        name.append(Fund_Name[CIK.index(item)])


In [17]:
outputdf=pd.DataFrame(name, columns=['name'])
outputdf['weblinks']=weblinks
outputdf['CIK_13F']=CIK_13F
outputdf.to_csv('CIK13F.csv')

In [18]:

date = []
info_table = []
info_type = []

In [19]:
for page in weblinks:
    html = urllib2.urlopen(page).read()
    
    date_index = html.find('div class=\"infoHead\">Filing Date<')
    date_html = html[date_index:]
    date_index2 = date_html.find('class=\"info\">')
    date_html = date_html[date_index2:]
    start = date_html.find(">")
    end = date_html.find("<")
    date.append(date_html[start+1:end])  # This gets us the date
    
    if html.find('td scope=\"row\">2<') != -1: # By this condition we check whether there is a xml version. For an xml version there are two items present in each table in the page
        
        index = html.find('td scope=\"row\">2<')
        html2 = html[index:]
    
        start = html2.find('a href=\"/Archives/edgar/data/')
        remaining = html2[start:]
        end = remaining.find('>')
    
        info_table.append(remaining[8:end-1]) # we collect the link
    
        info_type.append('xml') # we mention that this is an xml link
            
    else: # Or else, we get the idea that the info table is in text format
        
        index = html.find('td scope=\"row\">Complete submission text file')
        html2 = html[index:]
        
        start = html2.find('a href=\"/Archives/edgar/data/')
        remaining = html2[start:]
        end = remaining.find('>')
    
        info_table.append(remaining[8:end-1]) 

        info_type.append('text') # we mention that this is a text link

In [20]:
outputdf=pd.DataFrame(name, columns=['name'])
outputdf['date']=date
outputdf['info_type']=info_type
outputdf['info_table']=info_table
outputdf.to_csv('13Flink.csv')